In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy
import time
import pandas as pd
from sodapy import Socrata
results_dic = {}
list_of_depts = ['HPD', 'NYPD', 'DSNY', 'DOT', 'DOB', 'DPR', 'DHS' 'DFTA', 'DEP']
for key in list_of_depts:
# Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
#     client = Socrata("data.cityofnewyork.us", 'fCzaXZ3MLFDro2nWBiJ8CBYQh')

# # Example authenticated client (needed for non-public datasets):
# # client = Socrata(data.cityofnewyork.us,
# #                  MyAppToken,
# #                  userame="user@example.com",
# #                  password="AFakePassword")

# # First 2000 results, returned as JSON from API / converted to Python list of
# # dictionaries by sodapy.
#     results = client.get('fhrw-4uyv', where= f"Agency = '{key}'", limit = 100_000)
#     #print("Total number of non-null results: {}".format(meta_amount['cachedContents']['non_null']))
#     print("Number of results downloaded: {}".format(len(results)))
#     results[:3]
    
# Convert to pandas DataFrame
    results_dic[key] = pd.read_csv(f'./{key}.csv',dtype = object)
    print('brought in data')
    # Create a Dictionary to number each borough
    borough_dict = {'MANHATTAN': '1', 'BRONX' : '2', 'BROOKLYN': '3', 'QUEENS' : '4', 'STATEN' : '5'}
    # Map each borough to the desired number
    results_dic[key]['community_board_full'] = results_dic[key]['borough'].map(borough_dict)
        
    # Create a Community Board Number column
    results_dic[key]['CB_num']= [i.split(' ')[0] for i in results_dic[key].community_board]
    
    # Make all cells that are "Unspecified" into Null values
    for i in results_dic[key].columns:
        results_dic[key][i] = results_dic[key][i].map(lambda x: np.nan if x == 'Unspecified' else x)
        
    # Make all Community Board NUmber cells that are "0" into Null values
    results_dic[key].CB_num = results_dic[key].CB_num.map(lambda x: np.nan if x == '0' else x)    
    results_dic[key].dropna(subset = ['CB_num'],inplace = True)
    results_dic[key].dropna(subset = ['borough'],inplace = True)

    # Create a column with the borough number and the community board number 
    # Example: Community board "Brooklyn 11" is now Community Board "311"
    results_dic[key]['community_board_full'] = results_dic[key]['community_board_full'] + results_dic[key]['CB_num']


    # Turn all of the date columns into Datetime
    results_dic[key]['created_date'] = pd.to_datetime(results_dic[key]['created_date'])
    results_dic[key]['closed_date'] = pd.to_datetime(results_dic[key]['created_date'])
        
    # Create New columns for Month, Year, and a 3rd column with month and year together
    results_dic[key]['Year'] = results_dic[key]['created_date'].dt.year
    results_dic[key]['Month'] = results_dic[key]['created_date'].dt.month
    results_dic[key]['year-month'] = str(results_dic[key]['Year'])+'-'+str(results_dic[key]['Month'])

    # Create a column to see how long each case took to close
    results_dic[key]['time_to_close'] = results_dic[key].closed_date - results_dic[key].created_date

    # Create a column to see how many minutes the case took to close
    results_dic[key]['total_minutes'] = results_dic[key].time_to_close.map(lambda x: x.total_seconds()/60)
    
    
    results_dic[key] = results_dic[key].rename(columns={'complaint_type': key})
    # Create a new dataframe with dummy variables for each complaint type
#     exec(f"{key}_dummies = pd.get_dummies(results_dic[key].complaint_type)")
    

    df = results_dic[key].groupby(['Month','Year','community_board_full']).count().reset_index()[['Month','Year','community_board_full',key]]
    df.to_csv(f'{key}_to_noah.csv')
    print(f'cleaned {key} df')

brought in data


In [ ]:
for dept in list_of_depts[1:]:
    df = pd.merge(df,results_dic[dept].groupby(['Month','Year','community_board_full']).count().reset_index()[['Month','Year','community_board_full',dept]],how = 'right',on=['Month','Year','community_board_full'])
    print(f'Merged {dept}')